In [1]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
import pandas as pd
from sklearn.metrics import (
        make_scorer,
        confusion_matrix, 
        cohen_kappa_score, 
        accuracy_score, 
        precision_score, 
        recall_score, 
        f1_score, 
        roc_auc_score
)
from sklearn.model_selection import cross_validate, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier # decision trees for classification
from sklearn.neural_network import  MLPClassifier # neural networks for classification
from sklearn.naive_bayes import GaussianNB # naive bayes for classification
from sklearn.svm import SVC # support vector machines for classification

In [4]:
def specificity_score(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    return tn / (tn+fp)

In [5]:
METRICS = {
        "accuracy": make_scorer(accuracy_score),
        "precision": make_scorer(precision_score),
        "recall": make_scorer(recall_score),
        "f1": make_scorer(f1_score),
        "AUC": make_scorer(roc_auc_score, needs_proba=True),
        "specificity": make_scorer(specificity_score),
        "kappa":make_scorer(cohen_kappa_score)
}

In [6]:
# Carregar o conjunto de dados
d = pd.read_excel("C3_resultadofim.xlsx")
d

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,1843,1,1.8,1,13,1,16,0.7,134,6,...,118,580,3213,15,14,15,1,1,0,2
1,1424,0,0.5,1,16,1,23,0.6,80,4,...,1399,1663,1053,14,11,16,1,1,1,1
2,1844,1,2.3,1,1,0,51,0.7,158,8,...,533,882,1724,5,1,14,0,1,1,1
3,1501,0,1.1,0,12,1,2,0.2,187,1,...,198,1056,2487,13,12,16,1,1,1,2
4,1976,1,2.6,1,6,0,45,1.0,136,4,...,652,1933,2470,11,1,3,0,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1581,712,1,1.6,0,6,1,9,0.8,172,2,...,881,1234,3612,13,5,4,1,0,0,2
1582,1664,1,2.9,1,9,1,15,0.5,177,7,...,1178,1929,1968,12,7,9,1,0,0,2
1583,1375,1,1.1,0,3,1,9,0.9,145,7,...,475,726,3869,19,18,10,1,1,1,2
1584,1910,1,1.7,0,0,0,34,0.6,115,2,...,985,1284,309,8,4,10,0,0,1,1


In [7]:
X, y = d.drop("price_range", axis=1), d["price_range"]
d

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,1843,1,1.8,1,13,1,16,0.7,134,6,...,118,580,3213,15,14,15,1,1,0,2
1,1424,0,0.5,1,16,1,23,0.6,80,4,...,1399,1663,1053,14,11,16,1,1,1,1
2,1844,1,2.3,1,1,0,51,0.7,158,8,...,533,882,1724,5,1,14,0,1,1,1
3,1501,0,1.1,0,12,1,2,0.2,187,1,...,198,1056,2487,13,12,16,1,1,1,2
4,1976,1,2.6,1,6,0,45,1.0,136,4,...,652,1933,2470,11,1,3,0,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1581,712,1,1.6,0,6,1,9,0.8,172,2,...,881,1234,3612,13,5,4,1,0,0,2
1582,1664,1,2.9,1,9,1,15,0.5,177,7,...,1178,1929,1968,12,7,9,1,0,0,2
1583,1375,1,1.1,0,3,1,9,0.9,145,7,...,475,726,3869,19,18,10,1,1,1,2
1584,1910,1,1.7,0,0,0,34,0.6,115,2,...,985,1284,309,8,4,10,0,0,1,1


In [8]:
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=1234)

In [9]:
dt = DecisionTreeClassifier(max_depth=3, random_state=1234)
splitter = StratifiedKFold(10, random_state=1234, shuffle=True)
scores = cross_validate(dt, X, y, cv=splitter, scoring=METRICS)
dt_scores = pd.DataFrame(scores)
pd.DataFrame(dt_scores.mean()).T

,fit_time,score_time,test_accuracy,test_precision,test_recall,test_f1,test_AUC,test_specificity,test_kappa
0,0.008831,0.010404,0.922462,0.912485,0.935759,0.923191,0.967665,0.935759,0.844904
